In [2]:
import os
import clip.clip as clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
model, preprocess = clip.load('/data/datasets/Epickitchens100_clips/ViT-B-16.pt', device)

In [11]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f103b2859d0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [17]:
from torchvision.io import read_image
from torchvision import transforms
class EGO4D_center(Dataset):
    def __init__(self, data_path,anno_path, transform=None, target_transform=None):
        super(EGO4D_center, self).__init__()
        self.data_path = data_path
        self.anno_path = anno_path
        self.transform = transform
        self.target_transform= target_transform
        import pandas as pd
        annotations = pd.read_csv(self.anno_path, header=None, delimiter=',')
        self.img_name = list(annotations.values[:, 0])
        self.labels= list(annotations.values[:,2])

        self.transform = transforms.Compose([
         transforms.Resize(size=256),
        transforms.CenterCrop(size=(224, 224)),
        transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
    ])

        self.target_transform = transforms.Compose([
            transforms.Resize((224, 224)),      # 개와 고양이 사진 파일의 크기가 다르므로, Resize로 맞춰줍니다.
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # 이미지 정규화
        ])
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # file 경로
        img_path     = self.img_name[idx] + '.jpg'
        full_path    = os.path.join(self.data_path, img_path)
        # torchvision로 이미지 로드
        image = read_image(full_path)
        label = self.labels[idx]
        # transform 적용
        if self.transform:
            image = self.transform(image.float())
        if self.target_transform:
            image = self.target_transform(image.float())
        # image, label return
        return image, label

In [18]:
train_datasets=EGO4D_center('/local_datasets/ego4d_center/lta/train/','/data/datasets/ego4d_clips/lta_train.csv',transform=True)
val_datasets=EGO4D_center('/local_datasets/ego4d_center/lta/val/','/data/datasets/ego4d_clips/lta_val.csv',target_transform=True)



In [19]:
train_dataloader = DataLoader(train_datasets, batch_size=100, shuffle=True)
test_dataloader = DataLoader(val_datasets, batch_size=100, shuffle=True)

In [20]:
def get_features(dataset,dataloader):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [21]:
train_features, train_label = get_features(train_datasets,train_dataloader)

100%|██████████| 237/237 [00:51<00:00,  4.62it/s]


In [22]:
val_features, val_label = get_features(val_datasets,test_dataloader)


100%|██████████| 156/156 [00:32<00:00,  4.80it/s]


In [58]:
train_features.shape

(23610, 512)

loss_fn = nn.CrossEntropyLoss()

In [75]:
classifier = SGDClassifier('log_loss',learning_rate='optimal',max_iter=500)
classifier.fit(train_features, train_label)

SGDClassifier(loss='log_loss')

In [78]:
predictions = classifier.predict(val_features)
accuracy = np.mean((val_label == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 10.085


In [23]:
classifier = LogisticRegression(random_state=0, C=0.3, max_iter=1000, verbose=1)
classifier.fit(train_features, train_label)
predictions = classifier.predict(val_features)
accuracy = np.mean((val_label == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       206739     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.41635D+05    |proj g|=  7.47081D+03


 This problem is unconstrained.



At iterate   50    f=  8.41629D+04    |proj g|=  9.67964D+02

At iterate  100    f=  7.52519D+04    |proj g|=  2.25670D+02

At iterate  150    f=  7.31262D+04    |proj g|=  9.91822D+01

At iterate  200    f=  7.26244D+04    |proj g|=  4.38993D+01

At iterate  250    f=  7.25109D+04    |proj g|=  3.46552D+01

At iterate  300    f=  7.24842D+04    |proj g|=  8.58792D+00

At iterate  350    f=  7.24765D+04    |proj g|=  3.73085D+01

At iterate  400    f=  7.24726D+04    |proj g|=  7.34329D+00

At iterate  450    f=  7.24664D+04    |proj g|=  5.94346D+00

At iterate  500    f=  7.24509D+04    |proj g|=  2.18026D+01

At iterate  550    f=  7.24240D+04    |proj g|=  4.41468D+01

At iterate  600    f=  7.24010D+04    |proj g|=  1.98824D+01

At iterate  650    f=  7.23917D+04    |proj g|=  2.18140D+01

At iterate  700    f=  7.23895D+04    |proj g|=  4.17237D+00

At iterate  750    f=  7.23886D+04    |proj g|=  4.86445D+00

At iterate  800    f=  7.23879D+04    |proj g|=  8.21434D+00

At iter

/data/jong980812/anaconda3/envs/vmae2/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min finished
